In [1]:
open_r_api = "API KEY"

In [2]:
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.llms import ChatMessage

llm = OpenRouter(
    api_key=open_r_api,
    max_tokens=2048,
    temperature=0.7,
    context_window=4096,
    model="tngtech/deepseek-r1t-chimera:free",
)

In [4]:
user_input = "write a 3-sentence text with erroneous judgments about Science. Only text, no comments."

messages = [
    ChatMessage(role="system", content="You are a helpful AI assistant."),
    ChatMessage(role="user", content=user_input),
]

response = llm.chat(messages)
initial_response = response.message.content
print(initial_response)

Science proves absolute truths that never change over time, unlike other fields that rely on mere opinions. All scientific experiments always yield perfect results without any room for error or interpretation. Scientists never disagree with one another because the facts are always clear and unambiguous.


In [5]:
sys_prompt = """
Role: You are a professional critic with the characteristics of a person who doubts every statement. 
Task: add to each judgment next to the degree of your confidence in its correctness [an integer from 0 to 100].
Limitations:
- Do not add your own comments. Send the source text with the added ratings.
Format:
Entropy is always decreasing in isolated systems [30] which is why ice melts faster in warmer temperatures [20]...
"""

messages = [
    ChatMessage(role="system", content=sys_prompt),
    ChatMessage(role="user", content=f"Text: {initial_response}"),
]

response = llm.chat(messages)
analyzed_text = response.message.content
print(analyzed_text)

Science proves absolute truths that never change over time [10], unlike other fields that rely on mere opinions [15]. All scientific experiments always yield perfect results without any room for error or interpretation [5]. Scientists never disagree with one another because the facts are always clear and unambiguous [5].


In [10]:
json_sys_prompt = """
At the entrance, you receive a text that is divided into the ratings of each sentence. Output the json with the top-3 smallest values from the text.
Output Format:
{
    "judgment": "Entropy is always decreasing in isolated systems",
    "confidence": 30
},
"""

messages = [
    ChatMessage(role="system", content=json_sys_prompt),
    ChatMessage(role="user", content=f"Text: {analyzed_text}"),
]

response = llm.chat(messages)
text2json = response.message.content
print(text2json)

[
    {
        "judgment": "All scientific experiments always yield perfect results without any room for error or interpretation",
        "confidence": 5
    },
    {
        "judgment": "Scientists never disagree with one another because the facts are always clear and unambiguous",
        "confidence": 5
    },
    {
        "judgment": "Science proves absolute truths that never change over time",
        "confidence": 10
    }
]


In [11]:
message = ChatMessage(role="user", content=f"""{text2json} \n
edit the json and add a reflective question based on the confidence score to each one.
Output format: json code only.""")

response = llm.chat([message])
edit_json = response.message.content
print(edit_json)

```json
[
    {
        "judgment": "All scientific experiments always yield perfect results without any room for error or interpretation",
        "confidence": 5,
        "reflective_question": "Can you think of any scientific experiments where results were inconclusive or required reinterpretation over time?"
    },
    {
        "judgment": "Scientists never disagree with one another because the facts are always clear and unambiguous",
        "confidence": 5,
        "reflective_question": "What examples can you recall where scientific communities had prolonged debates about interpreting the same data?"
    },
    {
        "judgment": "Science proves absolute truths that never change over time",
        "confidence": 10,
        "reflective_question": "How would you reconcile this belief with historical cases like Newtonian physics being superseded by Einstein's relativity?"
    }
]
```


In [13]:
sys_prompt = """
You are a critical judgment analyst tasked with examining statements for their validity, identifying weak points, and formulating truthful reasoning. Your focus is analyzing linguistic constructions (words, phrases) for ambiguity, bias, logical fallacies, or insufficient argumentation.

Input data:
```json
{
    "judgment": "...",
    "confidence": int,
    "reflective_question": "...",
}
```

Instructions:

* Judgment Analysis
1. Identify vague, emotionally charged, or subjective phrasing.
2. Determine how well-justified they are (based on logic, data, context).

* For each critical element, generate three responses in different tones:
1. logical_answer: Objective analysis, reliance on data/logic.
2. empathic_answer: Empathy, support, emphasis on the author’s intent.
3. skeptic_answer: Doubt, questioning assumptions, alternative interpretations.

JSON format for each critical element:
```json
{
    "judgment": "...",
    "confidence": int,
    "reflective_question": "...",
    "logical_answer": "...",
    "empathic_answer": "...",
    "skeptic_answer": "..."
}
```
"""

messages = [
    ChatMessage(role="system", content=sys_prompt),
    ChatMessage(role="user", content=f"Text: {edit_json}"),
]

response = llm.chat(messages)
answers_json = response.message.content
print(answers_json)

```json
[
    {
        "judgment": "All scientific experiments always yield perfect results without any room for error or interpretation",
        "confidence": 5,
        "reflective_question": "Can you think of any scientific experiments where results were inconclusive or required reinterpretation over time?",
        "logical_answer": "This statement contradicts the fundamental principle of experimental error margins and peer review processes. The replication crisis in psychology and inconsistent results in cancer biology demonstrate that most experiments require statistical interpretation and repeated verification.",
        "empathic_answer": "I understand the appeal of imagining science as a flawless process - it reflects our human desire for certainty. However, the beauty of science lies precisely in its capacity to refine imperfect results through collaborative effort.",
        "skeptic_answer": "What evidence supports the claim that *all* experiments yield perfect results? H

In [16]:
message = ChatMessage(role="user", content=f"""{answers_json} \n
You need to edit the JSON and select 1-2 best answers for each critical element in the format of only "judgment":"best answer".
Output format: json code only.""")

response = llm.chat([message])
edit_json = response.message.content
print(edit_json)

```json
[
    {
        "judgment": "All scientific experiments always yield perfect results without any room for error or interpretation",
        "best_answer": [
            "This statement contradicts the fundamental principle of experimental error margins and peer review processes. The replication crisis in psychology and inconsistent results in cancer biology demonstrate that most experiments require statistical interpretation and repeated verification.",
            "I understand the appeal of imagining science as a flawless process - it reflects our human desire for certainty. However, the beauty of science lies precisely in its capacity to refine imperfect results through collaborative effort."
        ]
    },
    {
        "judgment": "Scientists never disagree with one another because the facts are always clear and unambiguous",
        "best_answer": [
            "This claim ignores well-documented controversies like the nature of dark matter, interpretations of quantum m